# Explore here

In [1]:
import os
import matplotlib.pyplot as plt
import cv2
from glob import glob


ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [ ]:
def plot_images(category, num_images=9):
    folder_path = f"./train/"
    images = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if category in img]
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)
        img = cv2.imread(images[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

# Plot Cats
plot_images('cat')

# Plot Dogs
plot_images('dog')


In [ ]:
IMG_SIZE = (200, 200)

def preprocess_images(image_paths):
    images = []
    labels = []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.resize(img, IMG_SIZE)
        images.append(img)
        labels.append(1 if 'dog' in img_path else 0)  # 1 for dog, 0 for cat
    return images, labels


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    './train',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    './train',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(200, 200, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPool2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPool2D((2, 2)),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPool2D((2, 2)),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPool2D((2, 2)),

    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[
        ModelCheckpoint('best_model.h5', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=3)
    ]
)


In [ ]:
#Evaluar
from tensorflow.keras.models import load_model
best_model = load_model('best_model.h5')


In [ ]:
test_data = train_datagen.flow_from_directory(
    './test',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary'
)

test_loss, test_acc = best_model.evaluate(test_data)
print(f"Test Accuracy: {test_acc}")


In [ ]:
#Guardar el modelo
best_model.save('dog_vs_cat_model.h5')
print("Model saved as dog_vs_cat_model.h5")
